## People Data Labs - Update Industry

- [ ] Support These Industries
- [ ] Update Label Only

Misc

- machinery
- import and export
- apparel & fashion
- research
- law practice
- accounting
- consumer goods
- religious institutions
- restaurants
- events services
- retail
- staffing and recruiting
- hospitality
- real estate
- museums and institutions
- computer games
- logistics and supply chain
- food production
- public safety
- mechanical or industrial engineering
- telecommunications
- dairy
- music
- consumer services
- paper & forest products
- architecture & planning
- wholesale
- law enforcement
- food & beverages
- international trade and development
- electrical/electronic manufacturing
- textiles
- facilities services
- farming
- industrial automation
- packaging and containers
- consumer electronics
- professional training & coaching
- mining & metals
- transportation/trucking/railroad
- legal services
- sports
- automotive
- leisure, travel & tourism
- oil & energy
- building materials
- nonprofit organization management
- translation and localization
- business supplies and equipment
- utilities
- chemicals
- furniture
- venture capital & private equity
- printing
- nanotechnology
- online media
- fund-raising
- warehousing
- graphic design
- wine and spirits
- publishing
- human resources
- arts and crafts
- veterinary
- security and investigations
- cosmetics
- government administration
- alternative medicine
- maritime
- public relations and communications
- individual & family services
- design
- media production
- photography
- airlines/aviation
- plastics
- performing arts
- executive office
- philanthropy
- glass, ceramics & concrete
- shipbuilding
- writing and editing
- luxury goods & jewelry
- package/freight delivery
- wireless
- fishery
- commercial real estate
- alternative dispute resolution
- market research
- libraries
- animation
- tobacco
- judiciary
- ranching
- railroad manufacture


In [ ]:
def industry(ppl_industry):
    industries = {
      "Developer Platform": [
        "computer software",
        "non-profit organization management",
        "internet",
        "information technology and services",
        "semiconductors",
        "information services",
        "computer hardware",
        "program development",
        "computer & network security",
        "computer networking"
      ],
      "Financial": [
        "medical practice",
        "medical devices",
        "health, wellness and fitness",
        "banking",
        "investment management",
        "capital markets",
        "insurance"
      ],
      "Healthcare": [
        "biotechnology",
        "investment banking",
        "financial services",
        "mental health care",
        "pharmaceuticals",
        "hospital & health care"
      ],
      "Education": [
        "education management",
        "e-learning",
        "higher education",
        "primary/secondary education"
      ],
      "Enterprise": [
        "management consulting"
      ],
      "Construction / Real Estate": [
        "civil engineering",
        "construction"
      ],
      "Entertainment": [
        "newspapers",
        "broadcast media",
        "entertainment",
        "gambling & casinos",
        "motion pictures and film",
        "recreational facilities and services",
        "fine art",
        "sporting goods"
      ],
      "Government": [
        "civic & social organization",
        "think tanks",
        "international affairs",
        "public policy",
        "government relations",
        "defense & space",
        "political organization",
        "legislative office",
        "military",
        "aviation & aerospace"
      ],
      "Sales / Marketing": [
        "marketing and advertising"
      ],
      "Climate Change": [
        "environmental services"
        "renewables & environment",
      ],
      "Agency / Consulting": [
        "outsourcing/offshoring"
      ],
    }
    
    for k, v in industries.items():
        if ppl_industry in v:
            return k
    return ''
        
assert industry('biotechnology') == 'Healthcare', "Need to be healthcare" 
assert industry('asdasd') == '', 'Needs to be empty'

In [ ]:
import requests
import sqlite3
import datetime

 
conn = sqlite3.connect('/Users/abhi/Dropbox/Data/PeopleDataLabs.db')
c = conn.cursor()

start = 0
stop = False
filter_id = 216

while not stop:
    response = requests.get("https://api.pipedrive.com/v1/organizations?api_token=484ecee3191b583e2c8ff9ddfc6b1872e02e7a64&limit=100&start={}&filter_id={}".format(start, filter_id))
    data = response.json()

    if data['additional_data']['pagination']['more_items_in_collection']:
        start = data['additional_data']['pagination']['next_start']
    else:
        stop = True

    for i in data['data']:
        linked_in = i['cf7bdfaa14c6382644ec04afa6045fb086c526e2']
        domain = i['5588b70e08a484430a0a5eab36022dc2905bcdf7']
        label = i['label']

        c.execute('''select "linkedin url", domain, industry, "current employee estimate" from PeopleDataLabsOrgs where name = "{}"  and country = "united states";'''.format(i['name'].lower()))
        data = c.fetchone()
        
        print(data)
        update_data = {
            '57cc06f77c9dfcd60dc43f34f19dbd5190c89093': datetime.timedelta(days=7)
        }
        
        if data is not None:
            if linked_in is None or linked_in is '':
                update_data['cf7bdfaa14c6382644ec04afa6045fb086c526e2'] = data[0]
                if update_data['cf7bdfaa14c6382644ec04afa6045fb086c526e2'] is None or update_data['cf7bdfaa14c6382644ec04afa6045fb086c526e2'] is '':
                    update_data['cf7bdfaa14c6382644ec04afa6045fb086c526e2'] = 'N/A'
                    
            if domain is None or domain is '' or domain == 'Pending':
                update_data['5588b70e08a484430a0a5eab36022dc2905bcdf7'] = data[1]
                if update_data['5588b70e08a484430a0a5eab36022dc2905bcdf7'] is None or update_data['5588b70e08a484430a0a5eab36022dc2905bcdf7'] is '':
                    update_data['5588b70e08a484430a0a5eab36022dc2905bcdf7'] = 'N/A'
                    
            if label is None or label is '':
                update_data['label'] = industry(data[2])
                if update_data['label'] is '':
                    del update_data['label']
                
        print("https://opszero.pipedrive.com/organization/{} / {} -> {}".format(i['id'], i['name'], update_data))
        resp = requests.put("https://api.pipedrive.com/v1/organizations/{}?api_token=484ecee3191b583e2c8ff9ddfc6b1872e02e7a64".format(i['id']), data=update_data)

# Insights Database

## KPIs


```
CREATE TABLE IF NOT EXISTS kpis (
  id SERIAL PRIMARY KEY,
  key varchar(100),
  config jsonb default {},
  created_at datetime default now(),
);
```

- [ ] Migrate DynamoDB to Insights DB
- [ ] Inserts Should be 

• DataPoint
◦ Category: Organization
◦ Source: “PeopleDataLab”
◦ Config: {}


• Create a machine within that VPC cloud9
• Copy data via psql
• Shutdown machine

In [ ]:
import aurora_data_api
import boto3
import json
import pandas as pd

df = pd.read_csv('7mm_companies.csv')

for i, row in df.iterrows():
    print([
        {
            'name': 'name',
            'value': {
                stringValue: row[0]
            }
        }
    ])
    break

def insights_insert_people_data_labs_orgs():
    cluster_arn = "arn:aws:rds:us-west-2:451305228097:cluster:insights"
    secret_arn = "arn:aws:secretsmanager:us-west-2:451305228097:secret:insights/dataapi-86PSRB"

    session = boto3.Session(profile_name='opszero')
    rds = session.client("rds-data")

#             cursor.execute("""
#     CREATE TABLE IF NOT EXISTS people_data_labs_organizations (
#             id SERIAL PRIMARY KEY,
#             name varchar(100),
#             domain varchar(100),
#             year_founded varchar(100),
#             industry varchar(100),
#             size_range varchar(100),
#             locality varchar(100),
#             county varchar(100),
#             linkedin varchar(100),
#             current_employee varchar(50),
#             total_employee varchar(50)
#     );
#     """)

#     response = client.batch_execute_statement(
#     database='insights',
#     parameterSets=[
#         [
#             {
#                 'name': 'string',
#                 'typeHint': 'DATE'|'DECIMAL'|'TIME'|'TIMESTAMP',
#                 'value': {
#                     'arrayValue': {
#                         'arrayValues': [
#                             {'... recursive ...'},
#                         ],
#                         'booleanValues': [
#                             True|False,
#                         ],
#                         'doubleValues': [
#                             123.0,
#                         ],
#                         'longValues': [
#                             123,
#                         ],
#                         'stringValues': [
#                             'string',
#                         ]
#                     },
#                     'blobValue': b'bytes',
#                     'booleanValue': True|False,
#                     'doubleValue': 123.0,
#                     'isNull': True|False,
#                     'longValue': 123,
#                     'stringValue': 'string'
#                 }
#             },
#         ],
#     ],
#     resourceArn='string',
#     schema='string',
#     secretArn='string',
#     sql='string',
#     transactionId='string'
# )

#             # name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate

#             with open('7mm_companies.json') as f:
#                 for cnt, line in enumerate(f):
#                     try:
#                         print("Line {}: {}".format(cnt, line))
#                         print(cursor.execute("insert into data_points (source, category, config) values ('PeopleDataLabs', 'Organization', '{}')".format(line),))
#                     except:
#                         pass
#                     if cnt == 100:
#                         conn.commit()

# insights_insert_people_data_labs_orgs()





# Sourcing

## Airtable Cleanup

- [ ] URL should be standardized before going into Pipedrive
- [ ] Send LinkedIN Contact Information
- [ ] Send Industry
- [ ] Company Size

## Built in Austin

In [ ]:
import json
import newspaper

jobs = newspaper.build('https://www.builtinaustin.com/jobs')
for article in jobs.articles:
    print(article.url)

print("Hi")
article = newspaper.Article('https://www.builtinaustin.com/job/engineer/senior-software-engineer/52751')
article.download()
article.parse()
print(article.text)
print(article.keywords)


## Docker Partners

- [ ] Add it into Pipedrive if it doesn't exist
- [ ] Type should be Partner

In [ ]:
from bs4 import BeautifulSoup
from cleanco import cleanco

html_doc = requests.get('https://www.docker.com/partners/find-partners').text
soup = BeautifulSoup(html_doc, 'html.parser')

for partner in soup.select('.partner-item'):
    for name in partner.select('h4'):
        print(cleanco(name.string).clean_name())


## GCP Partner Directory

https://cloud.withgoogle.com/partners/?partnerTypes=SPECIALIZATION_PARTNER,SERVICES_PARTNER&sort-type=RELEVANCE&capabilities=CLOUD_MIGRATION_CAPABILITY,INFRASTRUCTURE,APPLICATION_DEVELOPMENT_CAPABILITY&regions=NA_REGION
    
- [ ] Add into Pipedrive

In [ ]:
import requests
from bs4 import BeautifulSoup
from cleanco import cleanco

file = open('/Users/abhi/Dropbox/Data/Parse/gcppartner.html', 'r')
soup = BeautifulSoup(file.read(), 'html.parser')

for card in soup.select('.card'):
#     print(card.prettify())
    for name in card.select('h3'):
        print(cleanco(name.string).clean_name())


## AWS Partner



In [ ]:
import os  
import time
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options  

chrome_options = Options()  
#chrome_options.add_argument("--headless")  

driver = webdriver.Chrome(options=chrome_options)                                                                       
                                            

from bs4 import BeautifulSoup
from cleanco import cleanco

start = 0
end = 100
increment_size = 100

from airtable import Airtable
leads2 = Airtable('appjfIEfdbAa4oPFP', 'Leads2', api_key='keyBmpLT8XAxNLajb')

all_leads = []
while start <= end:
    print(start)

    driver.get(f"https://aws.amazon.com/partners/find/results/?size=100&start={start}&sort=Relevance&view=Grid")
    time.sleep(5)
    html_doc = driver.page_source
    soup = BeautifulSoup(html_doc, 'html.parser')

    all_leads += [{ 'Company Name': cleanco(partner.string).clean_name() } for partner in soup.select('.psf-partner-name')]    
    start += increment_size
    print(len(all_leads))
    
driver.close()

leads2.batch_insert(all_leads)        




# AngelList

Use AngelList to find new companies to reach out to.

Why:

AngelList is a great source of companies that we can use to reach out to.

Definition of Success:

Get a list of companies and add their Name and Url to the Leads Table
Use it to get a list of companies to reach out to.
Instruction Video:

Video of Task

Checklist:

Login to AngelList
Find Companies that are looking for the role Full Stack Engineer
Add them to Leads in Airtable
Organize

https://angel.co/jobs#find/f!%7B%22keywords%22%3A%5B%22devops%22%5D%2C%22locations%22%3A%5B%221688-United%20States%22%5D%2C%22company_size%22%3A%221-10%22%7D
https://angel.co/jobs#find/f!%7B%22keywords%22%3A%5B%22devops%22%5D%2C%22locations%22%3A%5B%221688-United%20States%22%5D%2C%22company_size%22%3A%2211-50%22%7D
Use Twitter: getopszero as the login
Use the links below to apply for a job.
Also fill out the form listed below Only Job Titles Containing: Site Reliability Engineer DevOps Cloud Engineer Add the company here: https://opszero.pipedrive.com/organizations/ Watch this video on what to do: https://www.dropbox.com/s/vir5yvitp8m5w66/save-prospects-into-pipedrive.mp4?dl=0 Fields to Find for Organization: Name URL Lifecycle: “Prospect” LinkedIn URL Funding Company Size Persona (If Company Size < 25 then it is Seed otherwise it is a Series A, Try to Figure out the closest industry.) Source: “AngelList Looking for DevOps” Status: “Orient” Further, add the AngelList link as a Note in the organization and make Source: “AngelList Looking for DevOps” TODO: Update this note for different personas. Note to Send: Hi, I know that finding DevOps Engineers is difficult so I was wondering if you would be interested in offloading your DevOps operations because opsZero can help build, manage and maintain your Cloud Infrastructure. We have extensive experience running high scalability infrastructure, helping companies scale from thousands of users to tens of millions of users. opsZero is trusted by many Y Combinator startups such as PicnichHealth, Lattice, Emburse and more to run their infrastructure. Our services are tailored specifically toward startups and small businesses who need to move fast and need a reliable infrastructure to go with it. I would love to chat with you further to get a better understanding of your DevOps challenges, and explain how using opsZero can help increase your productivity. I look forward to speaking with you soon. Yours, Abhi Yerra CEO opsZero https://www.opszero.com Platform Health Finance Enterprise Platform AngelList New Companies

Organize
For the following links: https://angel.co/done-deals/new-press https://angel.co/companies/trending/new

Go to Pipedrive Organizations Search to make sure that the organization doesn't exist. Fill out the following: Name URL Lifecycle: “Prospect” LinkedIn URL Funding Company Size Persona (If Company Size < 25 then it is Seed otherwise it is a Series A, Try to Figure out the closest industry) Source: “AngelList New Companies” Status: “Orient”


## Metrics
We need to keep track of the metrics of Approved to Rejected which should overtime get smaller and smaller. Right now it is a bit large so it is hard to track how we are improving over time.

 - [X] Track Approved / Rejected as a Percentage
 - [X] Track Worker, Assignment, Approve/Reject
 - [ ] Created At to Deleted At for Work. 

In [ ]:
from kpi import KPI

def block_bad_workers(kpi_name):
    KPI.increment(kpi_name, 'Approved', 0)
    KPI.increment(kpi_name, 'Rejected', 0)
    
    kpi = KPI.get(kpi_name)
    count = kpi.config['Approved'] + kpi.config['Rejected']

    if count > 10:
        success_rate = (1.0 - (kpi.config['Rejected'] / (kpi.config['Approved'] + kpi.config['Rejected']))) * 100
        print(success_rate)
        
        if success_rate < 80:
            print("Block")

block_bad_workers('SalesManager:Worker:AWJKSPG8RWZF4') # Really Bad
block_bad_workers('SalesManager:Worker:AXKHNWMKGT751') # Good
block_bad_workers('SalesManager:Worker:A2312VA97WKLWC') # No-op

## Sales and Marketing

https://airtable.com/tbls3ONmOTahmloG8/viwnNzQkjw1rQyp7V?blocks=hide

 - Remove unneeded sourcing entries which don't fulfill our target market on here.
 - We are not trying to expand internationally quite yet so we will filter these

# KPI

Sales Sourcing: KPI 95% Accuracy
Sales Orient: 30 Entires a Day

We will track this by everyday creating a Zapier task that updates when a task moves from Orient to Decide. 

Sales Decide: 30 Entries a Day

We track this by checking when it goes from Decide to Act

Sales Act: 10 Emails a day

We track this as a unique email and track it in KPI

Each of these will need to track overtime so it may make sense to use a Postgres RDS service powered by Serverless Aurora. 

KPI.increment_today(key, amount)

Add a secondary index on dynamodb. 


# Insights

- [ ] Track
- [ ] Websites
- [ ] Competitive Metrics
[ ] Companies 
[ ] Political Signals allows us to quickly get high quality data that allows us to make good high quality decisions over time. It is powered both by signal intelligence as well as human intelligence 

# Market Research

Primordial market research Figure out competitors Influencers Feature lists # Figure out what the Democratic Party's Data stuff is

# App / Sheepdog Signals for Pipedrive Notes a

Take Piperdive notes and a create signals to track

# AI

Have a way of using natural language processing to pull out entries in a text file.

Use that to make new projects and stories

https://aws.amazon.com/comprehend/features/
Use Keyphrase extraction


# Competitive Strategy

Create Observations based on the questions of the market and where we are * Run * Pick Organization * Stories * Stories auto\_expire\_at after a certain amount of time \-\> Automation # Prevent the creation of a retrospective story if it does not need to exist. # Add an automation to call RestApi with fields # Connect Slack to Sheepdog # Procedure type should be ProductProcedure Create a repeatable process for products. # App / Sharing common templates for organization. # Handle EventBridge events # How do I create custom workflows that are like process st. Need to likely move to yaml instead of json # App / WorkflowStory created from Procedure


# Apply AI to Pipedrive / Growth
Take training data. Anything marked as Act or Fail success and analyze it.

• Take label, startup size, predict best company to outreach. • Take based on previous learning orient to those companies

Means that we will need to quickly label data by people. Once they label the data we need to orient it. Only tasks marked as decide are important. They will cause tasks to be made more efficient over time.

Create a model to predict different companies segments using Ai quickly
• Take Act and Fail • Use those as the truths • Apply the model to the existing services • Outreach quickly. •

Add AI to Primitive

# Story Insights

Pull entries out of the text of retrospectives and reviews

# Etc

 - Public Kanban Define climate change tasks.
 - Need to collect data and potential fire zones 
 - Place to setup solar
 - Solar related companies
 - Government action
 - Climate groups and what they are already doing
 - Actionable tasks for climate change
 - Partner with climate groups and create technologies •


     If we knew what were doing it wouldnt be called research. - Albert Einstein

Competitive Landscape

 

Competitor data to collect

 

·        Link to their website.

·        "One-liners" and taglines competitors use.

·        Pricing and business model.

·        Blog/RSS link.

·        Social media links.

·        Key employees on social media.

o    Subscribe to RSS feeds of your competitors' blogs.

o    Follow your competitors and their key employees on social media.

o    Set up Google News alerts for competitors.

o    Monitor competitors' websites for changes.

·        Site monitoring tools

·        Site Alerts

·        Versionista

o    Subscribe to industry newsletters, magazines, trade journals etc.

o    Use Google Trends to do initial demand and search volume research (check out this article for specific tips)


Initially all of this will be directed for the Company and its products. So it should quickly allow me to make different decisions that are important to it. However we need to make certain components easily accessible such as data pulled to find good engineers. And we need to make other data about the world and make it actionable. We need to learn and summarize economic data and the changes in spending. We also need to look at changes in political situation.
Start with a Story of a single person using this idea and how they benefit. Why would they would choose this idea over something else?
What is the idea? What are the products & services we will need to provide? What is the value being generated? Who will be empowered by this idea?
How are we going to approach the task of bringing the idea to the world?
How is the idea the same or different from other companies who do this? How can they retaliate?
How does the idea fulfill our Vision?
What are the positions and procedures we need to do to be effective? Who are the partners we can work with? How can we use the Crowd?
What are the KPIs that need to be tracked?
If applicable a blog post/press release introducing the idea. Describe the Microcommit, the smallest feature set needed to test this idea. Who, What, Where, When, Why, How.
How is this idea Anti-Fragile? What are the risks?
Data
How will it make money?
How does this match with our competitive strategy? What competitive stage is the market? What is the industry analysis?
What is the First Principle breakdown of the idea? What are all the components that make it up? What is changing about each of the components in relation to time?
Collect
		
Competitors opsZero
Primitive
Startup Sonoma County
Market Trends	What technologies do people want? Jobs, etc.
What trends are we seeing in terms of what people want?
We need to start from where the critical mass of technology changes i.e Silicon Valley and how it coorolates to outside of the industry overtime.
Language Trends on Github	
Economic	How is the economy doing?
What it the average 	
Investments	Potential Purchases
Industry Analysis and Signals	
Political	Anti-Climate Candidates
	


Inbox for any signals associated with the organization we need to collect and run against
Research and signals
• Need to create a system to take in and crawl data. I need to basically be a web of data that just needs to be collected quickly.
• Crawlers that collect information about various businesses in Pipedrive and create additional notes.
• Crawlers that watch competitors and create signals on what is happening.
☐ Industry news sources need to be tracked within the app
• I need to make a system that lets me execute all at once.
• Who are the partners and vendors of a particular organization
• How can I grow the business much larger based on the outcome of a user.
• How can I apply AI to the Growth aspect of Sheepdog.
 
# - [ ] Public Data
# - [ ] Leadership / Culture
#   - Look at the documentation that is public
# - [ ] Competitive Strategy
# - [ ] Financial Data

Kickstarter posts
Crowdfunding
Figure out what companies could be potentially good to invest
• Figure out models for investment
• Analyze data from public
Help companies generate the crowdsourcing campaigns
Figure out which companies to invest in
Research
“The One Who Knocks”
Competitive Intelligence. We will do Research and Intelligence for companies including
Figure out what new technologies and research there are and put it to use.
Create Observers.
Figure out what is happening and ask the questions


Need to figure out high leverage data.
Not everything is high value so a task needs to have high value.




In [ ]:
import requests

API_KEY = "b7d7d42556631c2dc7687bf21235a986c07df4642b316c5835c6d24ba4a2f662"
PDL_VERSION = "v5"
PDL_URL = f"https://api.peopledatalabs.com/{PDL_VERSION}/person"
 
def find_by_email(email): 
    params = {
        "api_key": API_KEY,
        "email": email,
    }
    
    json_response = requests.get(PDL_URL, params=params).json()

    return (json_response['data']['first_name'], json_response['data']['last_name'],)


print(json_response['data']['first_name'])